In [6]:
import skimage.io as io
import time
import numpy as np
import cv2
import os

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from preprocessing import *
from LocalBinaryPattern import *
from ChainCodesFeatures import *
from PolygonFeatures import * 
from WindowFeatures import *
from windows import *

In [19]:
def Feature_Extractor(img):
    
    
    
    #img = cutHandWriting(img)
    
    img = cv2.resize(img,(img.shape[0]//4,img.shape[1]//4))
    
    #lbp = LocalBinaryPatterns(24,8)
    #lbp_features = lbp.describe(img)
    #st = time.time()
    edged = cv2.Canny(img, 30, 200)
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #en = time.time()
#     print("Contours : ",(en-st)*1000,"ms")
    
    #st = time.time()
    #windows, clusters, clusteredWindows = getClusteredWindows(img,13,10)
    #en = time.time()
#     print("Kesho : ",(en-st)*1000,"ms")
    
    #st = time.time()
    F5 = Chain_Codes_Feature_Extractor(img,contours,None)
    #en = time.time()
#     print("Ramzy : ",(en-st)*1000,"ms")
    
    #st = time.time()
    #f10, f11, f12, f13, f14 = PolygonFeatures(contours)
    #en = time.time()
#     print("Ibrahim Lefta : ",(en-st)*1000,"ms")
    
    #st = time.time()
    #Windows = list()
    
    #for i in range(clusteredWindows.shape[0]):
        #print(clusteredWindows[i])
        #Windows  += list(WindowFeatures(clusteredWindows[i].astype(np.uint8)))
    #en = time.time()
#     print("October : ",(en-st)*1000,"ms")
    return F5
    Features = list(F1)+list(F2)+list(F3)+list(F4)+list(F5)+list(F7)+list(F8)+list(F9)+list(f10)+list(f11)+list(f12)+list(f13)+list(f14)+list(lbp_features)+Windows
    
    

In [20]:
correct = []
f = open("correct.txt", "r")
for x in f:
    correct.append(int(x)) 

In [21]:
test_cases = os.listdir("TestData")
test_cases.sort()

In [22]:
i = 0 
acc = 0
for case in test_cases:
    writer1_1  = cv2.imread('TestData/'+case+"/1/1.png", cv2.IMREAD_GRAYSCALE)
    writer1_2  = cv2.imread('TestData/'+case+"/1/2.png", cv2.IMREAD_GRAYSCALE)
    writer2_1  = cv2.imread('TestData/'+case+"/2/1.png", cv2.IMREAD_GRAYSCALE)
    writer2_2  = cv2.imread('TestData/'+case+"/2/2.png", cv2.IMREAD_GRAYSCALE)
    writer3_1  = cv2.imread('TestData/'+case+"/3/1.png", cv2.IMREAD_GRAYSCALE)
    writer3_2  = cv2.imread('TestData/'+case+"/3/2.png", cv2.IMREAD_GRAYSCALE)
    test_img =  cv2.imread('TestData/'+case+"/test.png", cv2.IMREAD_GRAYSCALE)

    st = time.time()
    '''
    f1_1_5 , f1_1_11 , f1_1_13  = Feature_Extractor(writer1_1)
    f1_2_5 , f1_2_11 , f1_2_13  = Feature_Extractor(writer1_2)
    f2_1_5 , f2_1_11, f2_1_13  = Feature_Extractor(writer2_1)
    f2_2_5, f2_2_11 , f2_2_13 = Feature_Extractor(writer2_2)
    f3_1_5 , f3_1_11 , f3_1_13  = Feature_Extractor(writer3_1)
    f3_2_5 , f3_2_11 , f3_2_13  = Feature_Extractor(writer3_2)
    pred_f5 , pred_f11 , pred_f13  = Feature_Extractor(test_img)
    '''
    
    f1_1  = Feature_Extractor(writer1_1)
    f1_2  = Feature_Extractor(writer1_2)
    f2_1  = Feature_Extractor(writer2_1)
    f2_2 = Feature_Extractor(writer2_2)
    f3_1  = Feature_Extractor(writer3_1)
    f3_2  = Feature_Extractor(writer3_2)
    pred  = Feature_Extractor(test_img)
    
    features = np.array([f1_1,f1_2,f2_1,f2_2,f3_1,f3_2])
    #features_f5 = np.array([f1_1_5,f1_2_5,f2_1_5,f2_2_5,f3_1_5,f3_2_5])
    #features_f11 = np.array([f1_1_11,f1_2_11,f2_1_11,f2_2_11,f3_1_11,f3_2_11])
  # features_f11_f13 = np.array([list(f1_1_11)+list(f1_1_13),list(f1_2_11)+list(f1_2_13),list(f2_1_11)+list(f2_1_13),list(f2_2_11)+list(f2_2_13),list(f3_1_11)+list(f3_1_13),list(f3_2_11)+list(f3_2_13)])
    
    
    classifier = SVC()
    classifier.fit(features, [1,1,2,2,3,3])
    result = classifier.predict([pred])
    
    '''classifier_f11 = SVC()
    classifier_f11.fit(features_f11, [1,1,2,2,3,3])
    result_f11 = classifier_f11.predict([pred_f11])
    
    classifier_f11_f13 = SVC()
    classifier_f11_f13.fit(features_f11_f13, [1,1,2,2,3,3])
    result_f11_f13 = classifier_f11_f13.predict([list(pred_f11)+list(pred_f13)])
    result = np.array([result_f5[0],result_f11[0],result_f11_f13[0]])
    vote_1 = len(result[result==1])
    vote_2 = len(result[result==2])
    vote_3 = len(result[result==3])
    if vote_1 >= 2 :
        result = 1
    elif vote_2 >= 2 :
        result = 2
    elif vote_3 >=2 :
        result = 3
    else:
        result = result_f5
   '''
    
    
    en = time.time()
    print("Time Taken : ",(en-st))
    print(result,correct[i])
    if(result==correct[i]):
        acc+=1
    i+=1
    print(i,acc, "Accuracy : ",acc*100/i,"%")
    
    
    

Time Taken :  0.6448161602020264
[3] 3
1 1 Accuracy :  100.0 %
Time Taken :  0.7896945476531982
[3] 3
2 2 Accuracy :  100.0 %
Time Taken :  0.7780442237854004
[3] 3
3 3 Accuracy :  100.0 %
Time Taken :  0.738856315612793
[3] 3
4 4 Accuracy :  100.0 %
Time Taken :  0.7949955463409424
[3] 3
5 5 Accuracy :  100.0 %
Time Taken :  0.6779890060424805
[3] 3
6 6 Accuracy :  100.0 %
Time Taken :  0.7006943225860596
[3] 3
7 7 Accuracy :  100.0 %
Time Taken :  0.7494375705718994
[3] 3
8 8 Accuracy :  100.0 %
Time Taken :  0.7583572864532471
[3] 3
9 9 Accuracy :  100.0 %
Time Taken :  0.8332831859588623
[3] 3
10 10 Accuracy :  100.0 %
Time Taken :  0.7932529449462891
[3] 3
11 11 Accuracy :  100.0 %
Time Taken :  0.6707797050476074
[3] 3
12 12 Accuracy :  100.0 %
Time Taken :  0.632380485534668
[3] 3
13 13 Accuracy :  100.0 %
Time Taken :  0.7156062126159668
[3] 3
14 14 Accuracy :  100.0 %
Time Taken :  0.7331702709197998
[3] 3
15 15 Accuracy :  100.0 %
Time Taken :  0.6150269508361816
[3] 3
16 16 

# Results 
### SVM on F1 Feature : 95.7%  (68/71)
### SVM on F5 Feature : 97.2%  (69/71)
### SVM on F4 Feature : 97.2%  (69/71) With Resize /4  , 98.6% (70/71)  With Resize /2 
### SVM on F11 Feature : 94.3% (67/71)
### SVM on F11+F12+F13+F14   : 83.1% (59/71)
### SVM on  F11 + F13 : 93% (66/71)
###  3 SVM Classifiers   (F11+F13) , F11 , F5 : 95.7%  (68/71)

# Random Test


In [24]:
with open("forms.txt") as h:
    lines = h.readlines()[16:]

In [25]:
freq = np.zeros(680, dtype=int)
formWriter = {}
for i in lines:
    line = i.split(' ')
    key = line[1]
    if key in formWriter.keys():
        formWriter[key].append(line[0])
    else:
        formWriter[key] = [line[0]]

In [26]:
freq = [x for x, l in formWriter.items() if len(l)>2]

In [27]:
acc = 0
total = 0
for itr in range(500):
    keys = np.random.randint(len(freq), size=3)
    train = []
    test = []
    for k in keys:
        forms = formWriter[freq[k]]
        forms = [freq[k]+'-'+'{:03d}'.format(s)+'.jpg' for s in range(len(forms))]
        idxes = np.random.randint(len(forms), size=3)
        train.append(forms[idxes[0]])
        train.append(forms[idxes[1]])
        test.append(forms[idxes[2]])
    testWriter = np.random.randint(len(test))
    testImg = test[testWriter]
    
    trainFeatures = []
    Y = []
    for i in range(len(train)):
        img = cv2.imread('train/'+train[i], cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
       
        f = Feature_Extractor(img)
        trainFeatures.append(f)
        Y.append(i//2)
    trainFeatures =  np.array(trainFeatures)
    classifier = KNeighborsClassifier(n_neighbors=3)
   
    classifier.fit(trainFeatures, Y)
    
    img = cv2.imread("train/"+testImg, cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue

    testFeatures = Feature_Extractor(img)
    result = classifier.predict([testFeatures])
    if result[0] == testWriter:
        acc += 1
    total +=1
    print(len(testFeatures))
    print(total,acc, "Accuracy : ",acc*100/total,"%")
    print(result, testWriter)
    
    
    



512
1 1 Accuracy :  100.0 %
[0] 0
512
2 2 Accuracy :  100.0 %
[2] 2
512
3 2 Accuracy :  66.66666666666667 %
[0] 1
512
4 3 Accuracy :  75.0 %
[1] 1
512
5 4 Accuracy :  80.0 %
[2] 2
512
6 5 Accuracy :  83.33333333333333 %
[2] 2
512
7 6 Accuracy :  85.71428571428571 %
[1] 1
512
8 7 Accuracy :  87.5 %
[0] 0
512
9 8 Accuracy :  88.88888888888889 %
[2] 2
512
10 9 Accuracy :  90.0 %
[0] 0
512
11 10 Accuracy :  90.9090909090909 %
[1] 1
512
12 11 Accuracy :  91.66666666666667 %
[1] 1
512
13 12 Accuracy :  92.3076923076923 %
[1] 1
512
14 13 Accuracy :  92.85714285714286 %
[2] 2
512
15 14 Accuracy :  93.33333333333333 %
[0] 0
512
16 15 Accuracy :  93.75 %
[0] 0
512
17 16 Accuracy :  94.11764705882354 %
[2] 2
512
18 17 Accuracy :  94.44444444444444 %
[1] 1
512
19 18 Accuracy :  94.73684210526316 %
[2] 2
512
20 19 Accuracy :  95.0 %
[2] 2
512
21 20 Accuracy :  95.23809523809524 %
[0] 0
512
22 21 Accuracy :  95.45454545454545 %
[0] 0
512
23 22 Accuracy :  95.65217391304348 %
[0] 0
512
24 23 Accuracy

512
175 162 Accuracy :  92.57142857142857 %
[0] 0
512
176 163 Accuracy :  92.61363636363636 %
[1] 1
512
177 164 Accuracy :  92.65536723163842 %
[1] 1
512
178 165 Accuracy :  92.69662921348315 %
[1] 1
512
179 166 Accuracy :  92.73743016759776 %
[2] 2
512
180 167 Accuracy :  92.77777777777777 %
[1] 1
512
181 168 Accuracy :  92.81767955801105 %
[0] 0
512
182 169 Accuracy :  92.85714285714286 %
[2] 2
512
183 170 Accuracy :  92.89617486338798 %
[2] 2
512
184 170 Accuracy :  92.3913043478261 %
[1] 2
512
185 171 Accuracy :  92.43243243243244 %
[1] 1
512
186 172 Accuracy :  92.47311827956989 %
[1] 1
512
187 172 Accuracy :  91.97860962566845 %
[0] 1
512
188 173 Accuracy :  92.02127659574468 %
[2] 2
512
189 174 Accuracy :  92.06349206349206 %
[1] 1
512
190 175 Accuracy :  92.10526315789474 %
[0] 0
512
191 176 Accuracy :  92.14659685863874 %
[1] 1
512
192 177 Accuracy :  92.1875 %
[1] 1
512
193 178 Accuracy :  92.2279792746114 %
[0] 0
512
194 178 Accuracy :  91.75257731958763 %
[0] 1
512
195 179 

512
342 319 Accuracy :  93.27485380116958 %
[2] 2
512
343 320 Accuracy :  93.29446064139941 %
[2] 2
512
344 320 Accuracy :  93.02325581395348 %
[1] 0
512
345 321 Accuracy :  93.04347826086956 %
[1] 1
512
346 322 Accuracy :  93.0635838150289 %
[0] 0
512
347 322 Accuracy :  92.79538904899135 %
[1] 2
512
348 323 Accuracy :  92.816091954023 %
[2] 2
512
349 324 Accuracy :  92.83667621776505 %
[1] 1
512
350 325 Accuracy :  92.85714285714286 %
[0] 0
512
351 326 Accuracy :  92.87749287749288 %
[0] 0
512
352 327 Accuracy :  92.89772727272727 %
[1] 1
512
353 328 Accuracy :  92.91784702549575 %
[1] 1
512
354 329 Accuracy :  92.93785310734464 %
[1] 1
512
355 330 Accuracy :  92.95774647887323 %
[0] 0
512
356 331 Accuracy :  92.97752808988764 %
[0] 0
512
357 332 Accuracy :  92.99719887955182 %
[2] 2
512
358 333 Accuracy :  93.01675977653632 %
[1] 1
512
359 334 Accuracy :  93.03621169916434 %
[2] 2
512
360 335 Accuracy :  93.05555555555556 %
[0] 0
512
361 336 Accuracy :  93.07479224376732 %
[0] 0
512

# Results 
### SVM :  Normal Windows , No Ali part  : 86%    (86/100)     , 83%
### KNN :  Normal Windows , No Ali part  : 74%    (74/100)
### SVM :  Normal Windows , No Ali part no local features (84/100)
### SVM :  Normal Windwos , No Ali part , Pca  :64% (7/11)
### SVM : F1  90% (449/500)
### SVM : F5  97.2% (484/498)
### SVM : F11 90.6% (453/500)
### SVM : F4  95.2%  (474/498)
### KNN : F5 93.4%  (467/500)

In [ ]:
from Features.FeatureExtractor import *